In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.feature
import pandas as pd
import time
import math
from ast import literal_eval
%matplotlib inline

# Data Initialization

In [3]:
classes = ["adult_males", "subadult_males", "adult_females", "juveniles", "pups", "total"]

train_path = '/Users/YINAN/Local/Sea-lions/Data/Train/'
train_dotted_path = '/Users/YINAN/Local/Sea-lions/Data/TrainDotted/'

bad_images = [3,7,9,21,30,34,71,81,89,97,151,184,215,234,242,268,290,311,331,344,380,384,406,421,469,475,490,499,507,
              530,531,605,607,614,621,638,644,687,712,721]

file_names = [str(x) + '.jpg' for x in range(0,750) if x not in bad_images]
coordinates_df = pd.DataFrame(index=file_names, columns=classes)
# read all the coordinates 
coordinates_df = pd.read_csv("./coordinates.csv", index_col='index', converters={"total": literal_eval})

In [9]:
coordinates_df.head(5)

,adult_males,subadult_males,adult_females,juveniles,pups,total
index,,,,,,
0.jpg,"[(2353, 3726), (3638, 3539), (4720, 3476), (41...","[(1437, 3178), (4017, 3094), (5135, 2649), (63...","[(1953, 3730), (2882, 3728), (3443, 3725), (26...","[(3295, 3515), (1009, 3489), (3941, 3470), (10...","[(3143, 3732), (3486, 3726), (2694, 3711), (19...","[(3143, 3732), (1953, 3730), (2882, 3728), (34..."
1.jpg,"[(3012, 2880), (2233, 871)]","[(4655, 3056), (4574, 2992), (4609, 2959), (39...",[],"[(3619, 3609), (4781, 3414), (4437, 2933), (43...",[],"[(3619, 3609), (4781, 3414), (4655, 3056), (45..."
2.jpg,"[(2272, 1816), (2084, 1773)]",[],"[(2306, 1882), (2226, 1877), (2272, 1858), (23...","[(2342, 1887), (2332, 1880), (2037, 1860), (22...",[],"[(2342, 1887), (2306, 1882), (2332, 1880), (22..."
4.jpg,"[(2458, 3367), (60, 2535), (3088, 2503), (3347...","[(1693, 3505), (2143, 2954), (2966, 2923), (91...","[(675, 3479), (1093, 3465)]",[],[],"[(1693, 3505), (675, 3479), (1093, 3465), (245..."
5.jpg,"[(3952, 2911), (3053, 2483), (2996, 2154), (30...","[(3488, 3218), (3570, 1342), (3366, 770), (340...","[(3301, 1796), (3170, 1790), (3246, 1711), (30...","[(3214, 1769), (3188, 1705), (3147, 1684), (33...","[(3212, 1811), (3301, 1770), (3299, 1732), (32...","[(3488, 3218), (3952, 2911), (3053, 2483), (29..."


In [11]:
def extract_patches_with_sealions(coordinates_df):
    patches = []
    count = 0
    for filename in coordinates_df.index:
        count += 1
        image = cv2.imread(train_path + filename)
        for coordinates in coordinates_df.loc[filename].total:
            thumb = image[coordinates[1]-48:coordinates[1]+48,coordinates[0]-48:coordinates[0]+48,:]
            if np.shape(thumb) == (96, 96, 3):
                patches.append(cv2.cvtColor(thumb, cv2.COLOR_BGR2RGB))
        print("\r%d file completes, with total %d"%(count, len(coordinates_df)), end='')
    return patches

def extract_patches_without_sealions(coordinates_df):
    patches = []
    count = 0
    for filename in coordinates_df.index:
        sealion_coordinates_list = coordinates_df.loc[filename].total
        image = cv2.imread(train_path + filename)
        count += 1
        for row in range(image.shape[0]//96):
            for col in range(image.shape[1]//96):
                center = (row*96+48, col*96+48)
                flag = True
                for thumb in sealion_coordinates_list:
                    if math.sqrt((center[0] - thumb[1])**2 + (center[1] - thumb[0])**2) < math.sqrt(2)*96:
                        flag = False
                        break
                if flag:
                    patch_rgb = cv2.cvtColor(image[row*96:row*96+96, col*96:col*96+96], cv2.COLOR_BGR2RGB)
                    patches.append(patch_rgb)
        print("\r%d file completes, with total %d"%(count, len(coordinates_df)), end='')
    return patches 

# Pipeline

* positive patches extraction
* negative patches extraction
* train a simple logistic regression binary classifier
* testing

In [12]:
### extract pos patches
pos_num = 500
pos_df = pd.DataFrame(columns=["R", "G", "B"])
patches = extract_patches_with_sealions(coordinates_df=coordinates_df.sample(pos_num))

print("\n start extracting patches...")
for i in range(len(patches)):
    patch = patches[i]
    r = np.average(patch[:,:,0])
    g = np.average(patch[:,:,1])
    b = np.average(patch[:,:,2])
    pos_df = pos_df.append({'R':r, "G":g, "B":b}, ignore_index=True)
    print("\r%d patch completes, with total %d"%(i, len(patches)), end='')

pos_df['class'] = 1

500 file completes, with total 500

In [13]:
### extract neg patches
neg_num = 30
neg_df = pd.DataFrame(columns=["R", "G", "B"])
patches = extract_patches_without_sealions(coordinates_df=coordinates_df.sample(30))

print("\n start extracting patches...")
for i in range(len(patches)):
    patch = patches[i]
    r = np.average(patch[:,:,0])
    g = np.average(patch[:,:,1])
    b = np.average(patch[:,:,2])
    neg_df = neg_df.append({'R':r, "G":g, "B":b}, ignore_index=True)
    print("\r%d patch completes, with total %d"%(i, len(patches)), end='')

neg_df['class'] = 0

30 file completes, with total 30
 start extracting patches...
56997 patch completes, with total 56998

In [14]:
# construct a single data frame
total = pos_df.append(neg_df)

In [16]:
### build logistic regression 
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# Unfortunately I did not find a built-in method in sklearn to change the threshold of logistic regression
# In order to optimize recall, I need to decrease the threshold
def ClassifyWithThreshold(probabilities, threshold):
    return [+1 if x>=threshold else 0 for x in probabilities]

target = 'class'
variables = total.columns[total.columns != target]

x_train, x_test,y_train, y_test = \
model_selection.train_test_split(total[["B", "G", 'R']], total[target], test_size = 0.33, random_state=11)

logistic = linear_model.LogisticRegression(C=1e-7, random_state=1234)
logistic.fit(x_train, y_train)

y_pred = ClassifyWithThreshold(logistic.predict_proba(x_test)[:,1], 0.3)

print("recall score is {}".format(recall_score(y_test, y_pred)))
print("precision score is {}".format(precision_score(y_test, y_pred)))

recall score is 0.9928194700387621
precision score is 0.45470155117720673


# Test the module with a new data set

In [17]:
### extract pos patches
pos_num = 10
pos_df = pd.DataFrame(columns=["R", "G", "B"])
patches = extract_patches_with_sealions(coordinates_df=coordinates_df.sample(pos_num))

print("\n start extracting patches...")
for i in range(len(patches)):
    patch = patches[i]
    r = np.average(patch[:,:,0])
    g = np.average(patch[:,:,1])
    b = np.average(patch[:,:,2])
    pos_df = pos_df.append({'R':r, "G":g, "B":b}, ignore_index=True)
    print("\r%d patch completes, with total %d"%(i, len(patches)), end='')

pos_df['class'] = 1

10 file completes, with total 10
 start extracting patches...
493 patch completes, with total 494

In [18]:
### extract neg patches
neg_num = 10
neg_df = pd.DataFrame(columns=["R", "G", "B"])
patches = extract_patches_without_sealions(coordinates_df=coordinates_df.sample(30))

print("\n start extracting patches...")
for i in range(len(patches)):
    patch = patches[i]
    r = np.average(patch[:,:,0])
    g = np.average(patch[:,:,1])
    b = np.average(patch[:,:,2])
    neg_df = neg_df.append({'R':r, "G":g, "B":b}, ignore_index=True)
    print("\r%d patch completes, with total %d"%(i, len(patches)), end='')

neg_df['class'] = 0

30 file completes, with total 30
 start extracting patches...
58391 patch completes, with total 58392

In [19]:
total = pos_df.append(neg_df)

In [24]:
target = 'class'
variables = total.columns[total.columns != target]
y = total[target]
y_pred = ClassifyWithThreshold(logistic.predict_proba(total[variables])[:,1], 0.3)

print("recall score is {}".format(recall_score(y, y_pred)))
print("precision score is {}".format(precision_score(y, y_pred)))

recall score is 0.9878542510121457
precision score is 0.008553300382094156
